In [50]:
#!pip install geopy

In [110]:
#!pip install geocoder

In [111]:
import numpy as np
import pandas as pd

import io
from geopy.geocoders import Nominatim
import geocoder

In [16]:
data = pd.read_excel('Data_Train_6sep21.xlsx')
data_orig = data.copy()

In [5]:
data.shape

(11094, 9)

In [4]:
data.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11094 entries, 0 to 11093
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Restaurant     11094 non-null  object
 1   Location       11094 non-null  object
 2   Cuisines       11094 non-null  object
 3   Average_Cost   11094 non-null  object
 4   Minimum_Order  11094 non-null  object
 5   Rating         11094 non-null  object
 6   Votes          11094 non-null  object
 7   Reviews        11094 non-null  object
 8   Delivery_Time  11094 non-null  object
dtypes: object(9)
memory usage: 780.2+ KB


### Define Functions 

In [13]:
def combinfo(data) :
    combi = pd.DataFrame(data.nunique(),columns=['Nunique()'])
    combi2 = pd.DataFrame(data.isna().sum(),columns=['isna() count'])
    combi3 = pd.concat([combi, combi2],axis=1)

    #import io
    buffer = io.StringIO()
    data.info(buf=buffer, memory_usage=False)
    s = buffer.getvalue()
    df = pd.DataFrame(s.split("\n"),columns=['info'])
    df_info = df.loc[5:len(data.columns)+5-1,'info'].str.split(n=4,expand=True).reset_index(drop=True)
    df_info.drop(columns=[0,3],axis=1,inplace=True)
    df_info.columns = ['Column','Non-Null count','Dtype']
    df_info.set_index('Column',inplace=True)

    combi5 = pd.concat([combi3, df_info],axis=1)
    return combi5

In [8]:
def null_percent(data) :
    print("for all features")
    [print(data[i].isna().sum(),"(",round((data[i].isnull().sum()/len(data[i]))*100,2),"%)-->",i) \
     for i in data.columns if data[i].isna().sum()>0]

In [9]:
def vc(data, max_count,sort_idx) :
    [print(data[i].value_counts().sort_index()) for i in data.columns if data[i].nunique() < max_count]

In [10]:
null_percent(data)

for all features


In [14]:
combinfo(data)

,Nunique(),isna() count,Non-Null count,Dtype
Column,,,,
Restaurant,7480,0,11094,object
Location,35,0,11094,object
Cuisines,2179,0,11094,object
Average_Cost,26,0,11094,object
Minimum_Order,18,0,11094,object
Rating,33,0,11094,object
Votes,1103,0,11094,object
Reviews,761,0,11094,object
Delivery_Time,7,0,11094,object


In [15]:
vc(data,40,True)

BTM Layout 1, Electronic City                        22
Babarpur, New Delhi, Delhi                          282
Chandni Chowk, Kolkata                              126
Chatta Bazaar, Malakpet, Hyderabad                   54
D-Block, Sector 63, Noida                           806
Delhi Administration Flats, Timarpur                472
Delhi Cantt.                                        326
Delhi High Court, India Gate                        411
Delhi University-GTB Nagar                          545
Dockyard Road, Mumbai CST Area                      291
FTI College, Law College Road, Pune                 584
Gora Bazar, Rajbari, North Dumdum, Kolkata           79
Hyderabad Public School, Begumpet                    81
Jaya Nagar, Saidabad, Hyderabad                      48
Laxman Vihar Industrial Area, Sector 3A, Gurgoan    493
MG Road, Pune                                       402
Majestic                                            334
Mico Layout, Stage 2, BTM Layout,Bangalore      

In [17]:
data['Delivery_Time'] = pd.to_numeric(data['Delivery_Time'].str.replace('[^0-9]',''))

In [21]:
data['Rating'] = pd.to_numeric(data['Rating'].apply(lambda x : np.nan if x in ['Temporarily Closed','Opening Soon','-','NEW'] else x))

In [24]:
data['Minimum_Order'] = pd.to_numeric(data['Minimum_Order'].str.replace('₹',''))

In [27]:
data['Average_Cost'] = pd.to_numeric(data['Average_Cost'].str.replace('[^0-9]',''))

In [44]:
data['Votes'] = pd.to_numeric(data['Votes'].apply(lambda x : np.nan if x=='-' else x))
data['Reviews'] = pd.to_numeric(data['Reviews'].apply(lambda x : np.nan if x=='-' else x))

In [140]:
data['Location1'] = data['Location']

In [148]:
#data.to_csv('ws1.csv')
data = pd.read_csv('ws1.csv')

In [146]:
#data['Location1'] = data.Location1.replace(r'(^.*delhi.*$)', 'Delhi', r'(^.*mumbai.*$)','Mumbai', r'(^.*pune.*$)','Pune')
searchfor = ['pune','mumbai','delhi','kolkata','hyderabad','noida','gurgoan','bangalore']
# s[s.str.contains('|'.join(searchfor))]
data['Loc2'] = if data['Location1'].str.lower().str.contains('|'.join(searchfor)) == True : searchfor else data['Location1']

SyntaxError: invalid syntax (<ipython-input-146-b207cef59a97>, line 4)

In [150]:
data['Cuisines1'] = data['Cuisines'].str.split('[,]')
data = data.explode('Cuisines1').reset_index(drop=True)
cols = list(data.columns)
cols.append(cols.pop(cols.index('Restaurant')))
data = data[cols]

In [153]:
data.to_csv('ws2.csv',index=False)

In [152]:
data.head(10)

,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,Location1,City,Cuisines1,Restaurant
0,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200.0,50,3.5,12.0,4.0,30,"FTI College, Law College Road, Pune",Pune,Fast Food,ID_6321
1,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200.0,50,3.5,12.0,4.0,30,"FTI College, Law College Road, Pune",Pune,Rolls,ID_6321
2,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200.0,50,3.5,12.0,4.0,30,"FTI College, Law College Road, Pune",Pune,Burger,ID_6321
3,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200.0,50,3.5,12.0,4.0,30,"FTI College, Law College Road, Pune",Pune,Salad,ID_6321
4,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200.0,50,3.5,12.0,4.0,30,"FTI College, Law College Road, Pune",Pune,Wraps,ID_6321
5,"Sector 3, Marathalli","Ice Cream, Desserts",100.0,50,3.5,11.0,4.0,30,"Sector 3, Marathalli",Bangalore,Ice Cream,ID_2882
6,"Sector 3, Marathalli","Ice Cream, Desserts",100.0,50,3.5,11.0,4.0,30,"Sector 3, Marathalli",Bangalore,Desserts,ID_2882
7,Mumbai Central,"Italian, Street Food, Fast Food",150.0,50,3.6,99.0,30.0,65,Mumbai Central,Mumbai,Italian,ID_1595
8,Mumbai Central,"Italian, Street Food, Fast Food",150.0,50,3.6,99.0,30.0,65,Mumbai Central,Mumbai,Street Food,ID_1595
9,Mumbai Central,"Italian, Street Food, Fast Food",150.0,50,3.6,99.0,30.0,65,Mumbai Central,Mumbai,Fast Food,ID_1595


In [29]:
vc(data,40,True)

BTM Layout 1, Electronic City                        22
Babarpur, New Delhi, Delhi                          282
Chandni Chowk, Kolkata                              126
Chatta Bazaar, Malakpet, Hyderabad                   54
D-Block, Sector 63, Noida                           806
Delhi Administration Flats, Timarpur                472
Delhi Cantt.                                        326
Delhi High Court, India Gate                        411
Delhi University-GTB Nagar                          545
Dockyard Road, Mumbai CST Area                      291
FTI College, Law College Road, Pune                 584
Gora Bazar, Rajbari, North Dumdum, Kolkata           79
Hyderabad Public School, Begumpet                    81
Jaya Nagar, Saidabad, Hyderabad                      48
Laxman Vihar Industrial Area, Sector 3A, Gurgoan    493
MG Road, Pune                                       402
Majestic                                            334
Mico Layout, Stage 2, BTM Layout,Bangalore      

In [46]:
combinfo(data)

,Nunique(),isna() count,Non-Null count,Dtype
Column,,,,
Restaurant,7480,0,11094,object
Location,35,0,11094,object
Cuisines,2179,0,11094,object
Average_Cost,25,1,11093,float64
Minimum_Order,18,0,11094,int64
Rating,29,1963,9131,float64
Votes,1102,2074,9020,float64
Reviews,760,2312,8782,float64
Delivery_Time,7,0,11094,int64


In [43]:
data['Votes'].value_counts().index

Index(['-', '4', '6', '7', '9', '5', '8', '11', '12', '10',
       ...
       '1398', '750', '953', '1208', '5187', '4772', '3289', '457', '1419',
       '3633'],
      dtype='object', length=1103)

In [108]:
geolocator = Nominatim(user_agent="tutorial")# user_agent="specify_your_app_name_here")
location = geolocator.geocode('chennai 600007', addressdetails=True, namedetails=True)

print(type(location))
#print(location.raw['address']['village'])
print(location)
print(type(location.raw))

<class 'NoneType'>
None


AttributeError: 'NoneType' object has no attribute 'raw'

In [91]:
import json
import pprint
import urllib

LOCATION = 'https://api.qualifiedaddress.com/street-address/'
QUERY_STRING = urllib.urlencode({ # entire query sting must be URL-Encoded
    'auth-token': r'YOUR_API_KEY_HERE',
    'street': '1 infinite loop cupertino ca 95014'
})
URL = LOCATION + '?' + QUERY_STRING

response = urllib.urlopen(URL).read()
structure = json.loads(response)
pprint.pprint(structure)

In [134]:
g = geocoder.arcgis('Balaiyah Garden first cross street, tamil nadu', maxRows=20)
print(g.json)

{'address': 'K C Garden 1st Cross Street, Peravallur, Chennai, Tamil Nadu, 600082', 'bbox': {'northeast': [13.119690039552141, 80.23280505737281], 'southwest': [13.117690039552143, 80.2308050573728]}, 'confidence': 9, 'lat': 13.118690039552142, 'lng': 80.23180505737281, 'ok': True, 'quality': 'StreetName', 'raw': {'name': 'K C Garden 1st Cross Street, Peravallur, Chennai, Tamil Nadu, 600082', 'extent': {'xmin': 80.2308050573728, 'ymin': 13.117690039552143, 'xmax': 80.23280505737281, 'ymax': 13.119690039552141}, 'feature': {'geometry': {'x': 80.23180505737281, 'y': 13.118690039552142}, 'attributes': {'Score': 85.15, 'Addr_Type': 'StreetName'}}}, 'score': 85.15, 'status': 'OK'}
